# 3D nii to 2D png

In [2]:
import numpy as np
import os
from PIL import Image
import nibabel as nib

def nii2png(niifile, outdir):
    img = nib.load(niifile)
    data = img.get_fdata()

    min_val = data.min()
    max_val = data.max()
    scale = 255.0 / (max_val - min_val)

    for i in range(data.shape[2]):
        slice = data[:, :, i]
        slice = (slice - min_val) * scale
        slice = slice.astype(np.uint8)
        slice = Image.fromarray(slice, 'L')
        slice.save(os.path.join(outdir, f'{i}.png'))

    # Save scaling information for later use
    with open(os.path.join(outdir, 'scale_info.txt'), 'w') as f:
        f.write(f"{min_val}\n{max_val}\n")

    # Save the affine matrix
    affine = img.affine
    np.savetxt(os.path.join(outdir, 'affine_matrix.txt'), affine)

def png2nii(pngdir, outfile):
    files = [f for f in os.listdir(pngdir) if f.endswith('.png')]
    files.sort(key=lambda x: int(x[:-4]))
    slices = []
    for file in files:
        slice = np.array(Image.open(os.path.join(pngdir, file)))
        slices.append(slice)

    # Retrieve scaling information
    with open(os.path.join(pngdir, 'scale_info.txt'), 'r') as f:
        min_val, max_val = [float(line.strip()) for line in f]
    scale = 255.0 / (max_val - min_val)

    slices = np.array(slices)
    slices = (slices / scale) + min_val

    # Reorder the dimensions to match the original NIfTI file
    slices = np.transpose(slices, (1, 2, 0))

    # Load the affine matrix
    affine = np.loadtxt(os.path.join(pngdir, 'affine_matrix.txt'))

    img = nib.Nifti1Image(slices, affine)
    nib.save(img, outfile)


# test nii2png and png2nii

In [4]:
nii2png('./images/00001_AN001_V1_WIP RARE_baseline_7_analyze75.nii', './test/nii2png/test1')
png2nii('./test/nii2png/test1', './test/png2nii/temp.nii')

In [5]:
# 比较生成的nii和原始nii
img1 = nib.load('./images/00001_AN001_V1_WIP RARE_baseline_7_analyze75.nii')
img2 = nib.load('./test/png2nii/temp.nii')
data1 = img1.get_fdata()
data2 = img2.get_fdata()
print(data1.shape)
print(data2.shape)

(512, 512, 20)
(512, 512, 20)


In [6]:
nii2png('./dataset/1.5T-GE/imagesTr/data_001_0000.nii.gz', './test/nii2png/test_ge')
png2nii('./test/nii2png/test_ge', './test/png2nii/test_ge.nii')

In [7]:
img1 = nib.load('./dataset/1.5T-GE/imagesTr/data_001_0000.nii.gz')
img2 = nib.load('./test/png2nii/test_ge.nii')
data1 = img1.get_fdata()
data2 = img2.get_fdata()
print(data1.shape)
print(data2.shape)

(512, 512, 32)
(512, 512, 32)


In [8]:
nii2png('./dataset/1.5T-Phillips/imagesTr/data_001_0000.nii.gz', './test/nii2png/test_phillips')
png2nii('./test/nii2png/test_phillips', './test/png2nii/test_phillips.nii')

In [9]:
img1 = nib.load('./dataset/1.5T-Phillips/imagesTr/data_001_0000.nii.gz')
img2 = nib.load('./test/png2nii/test_phillips.nii')
data1 = img1.get_fdata()
data2 = img2.get_fdata()
print(data1.shape)
print(data2.shape)

(512, 512, 24)
(512, 512, 24)


In [10]:
nii2png('./dataset/3T-Achieva/imagesTr/data_001_0000.nii.gz', './test/nii2png/test_achieva')
png2nii('./test/nii2png/test_achieva', './test/png2nii/test_achieva.nii')

In [11]:
img1 = nib.load('./dataset/3T-Achieva/imagesTr/data_001_0000.nii.gz')
img2 = nib.load('./test/png2nii/test_achieva.nii')
data1 = img1.get_fdata()
data2 = img2.get_fdata()
print(data1.shape)
print(data2.shape)

(512, 512, 20)
(512, 512, 20)


In [12]:
nii2png('./dataset/3T-Ingenia/imagesTr/data_001_0000.nii.gz', './test/nii2png/test_ingenia')
png2nii('./test/nii2png/test_ingenia', './test/png2nii/test_ingenia.nii')

In [13]:
img1 = nib.load('./dataset/3T-Ingenia/imagesTr/data_001_0000.nii.gz')
img2 = nib.load('./test/png2nii/test_ingenia.nii')
data1 = img1.get_fdata()
data2 = img2.get_fdata()
print(data1.shape)
print(data2.shape)

(512, 512, 20)
(512, 512, 20)


# batch nii2png and png2nii

In [21]:
import os

def batch_nii2png(src_dir, dest_dir):
    """Convert all NIfTI files in src_dir to PNG sequences in dest_dir."""
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    for filename in os.listdir(src_dir):
        if filename.endswith(".nii"):
            nii_path = os.path.join(src_dir, filename)
            outdir = os.path.join(dest_dir, filename.split('.')[0])

            if not os.path.exists(outdir):
                os.makedirs(outdir)

            nii2png(nii_path, outdir)

def batch_png2nii(src_dir, dest_dir):
    """Convert all PNG sequences in src_dir to NIfTI files in dest_dir."""
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    for folder in os.listdir(src_dir):
        folder_path = os.path.join(src_dir, folder)
        if os.path.isdir(folder_path):
            nii_filename = folder + ".nii"
            nii_path = os.path.join(dest_dir, nii_filename)

            png2nii(folder_path, nii_path)

In [22]:
# Example usage
batch_nii2png('./images', './test/png_folder')
batch_png2nii('./test/png_folder', './test/nii_folder')